## Rating Prediction Project - Web Scraping

### Problem Statement

We have a client who has a website where people write different reviews for technical products. Now they are adding a new feature to their website i.e. The reviewer will have to add stars(rating) as well with the review. The rating is out 5 stars and it only has 5 options available 1 star, 2 stars, 3 stars, 4 stars, 5 stars. Now they want to predict ratings for the reviews which were written in the past and they don’t have a rating. So, we have to build an application which can predict the rating by seeing the review.

#### Data Collection Phase

You have to scrape at least 20000 rows of data. You can scrape more data as well, it’s up to you. more the data better the model

In this section you need to scrape the reviews of different laptops, Phones, Headphones, smart watches, Professional Cameras, Printers, Monitors, Home theater, Router from different e-commerce websites.

Basically, we need these columns-
1) reviews of the product.

2) rating of the product.

You can fetch other data as well, if you think data can be useful or can help in the project. It completely depends on your imagination or assumption.

### Scraping the data from Flipkart and Amazon

In [1]:
#Importing required libraries

import selenium
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

### Scraping Ratings and Reviews from Flipkart

In [2]:
driver=webdriver.Chrome("chromedriver.exe")
driver.maximize_window()
time.sleep(3)

In [3]:
url = "https://www.flipkart.com/"
driver.get(url)
driver.find_element_by_xpath("//div[@class='_2QfC02']/button").click()

In [4]:
srch_items = ['laptops', 'Phones','smart watches','Monitors']

In [5]:
title = []
review_text = []
ratings = []

In [6]:
def scrap():    
        for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):
            review_text.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"):
            title.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return

In [7]:
urls=[]

In [8]:
for i in srch_items:
    
    #Locating search bar
    
    srchBar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    
    #clicking on search button
    
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    page = []
    for i in driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))

In [9]:
len(urls)

384

In [10]:
for i in urls:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
        
        #clicking on all reviews
    
    try:
        btn=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass

    scrap()        
    try:
        n_page=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrap()
    except: continue

In [11]:
len(ratings), len(review_text), len(title)

(30681, 30681, 30681)

In [12]:
#creating dataframe

data = list(zip(title,review_text,ratings))
df2 = pd.DataFrame(data, columns = ["Review_Title","Reiew_Text","Ratings"])
df2

Review_Title  \
0              Worth every penny   
1                        Awesome   
2                  Great product   
3                      Wonderful   
4           Good quality product   
...                          ...   
30676            Waste of money!   
30677                       Nice   
30678        Could be way better   
30679                  Just wow!   
30680  Did not meet expectations   

                                              Reiew_Text Ratings  
0      Laptop is worth the price. It is fast and smoo...       5  
1                        Value for money just go for it✨       5  
2                                                   good       5  
3      It's good product but some times it gets stuck...       4  
4                       Great Laptop. You can go for it.       4  
...                                                  ...     ...  
30676                     It is worse screen was damaged       1  
30677                                               Good       3  
30678                                    Poor quality...       2  
30679                                       Good product       5  
30680                                           Very bad       1  

[30681 rows x 3 columns]

In [13]:
#saving the data into .csv file

df2.to_csv("flipkrt1_reviews.csv")

### Scraping Ratings and Reviews from Amazon

In [2]:
driver=webdriver.Chrome("chromedriver.exe")
driver.maximize_window()
time.sleep(3)

In [3]:
url = "https://www.amazon.in/"
driver.get(url)
time.sleep(3)

In [4]:
srch_items = ['laptops','phones','headphones','smart watches', 'Professional Cameras', 'Printers', 'Monitors', 'Home theater', 'Router']

In [5]:
title = []
review_text = []
ratings = []

In [6]:
for i in srch_items:
    srchbar = driver.find_element_by_id("twotabsearchtextbox")
    srchbar.send_keys(i)
    driver.find_element_by_id("nav-search-submit-button").click()  #clicking on search button
    time.sleep(1)
    item_url = []
    for i in driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        item_url.append(i.get_attribute('href'))
    for i in item_url:
        driver.get(i)
        time.sleep(1)
        for _ in range(2):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(1)
        
        #clicking on all reviews
        
        try:
            driver.find_element_by_xpath("//a[@class='a-link-emphasis a-text-bold']").click()
        except:
            pass
        
        url_ = []
        try:
            two_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[4]/td[2]/a")
            url_.append(two_str.get_attribute('href'))
        except: pass
        try:
            three_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[3]/td[2]/a")
            url_.append(three_str.get_attribute('href'))
        except: pass
        try:
            one_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[5]/td[2]/a")
            url_.append(one_str.get_attribute('href'))
        except: pass
        try:
            fiv_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[1]/td[2]/a")
            url_.append(fiv_str.get_attribute('href'))
        except: pass
        try:
            four_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[2]/td[2]/a")
            url_.append(four_str.get_attribute('href'))
        except: pass
        
        for i in url_:
            driver.get(i)
            time.sleep(1)
            for i in range(10):
                ids = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none review-views celwidget']/div")
                comment_ids = []
                for i in ids:
                    comment_ids.append(i.get_attribute('id'))

                for x in comment_ids:
                    
                    #Extracting user ids on page
                    
                    try:
                        review_title = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[2]/span[1]')
                        title.append(review_title.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        title.append('')

                    try:
                        rating = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[1]/i/span[1]')
                        ratings.append(rating.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        ratings.append('')

                    try:
                        text = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[4]/span/span')
                        review_text.append(text.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        review_text.append('')
                try:
                    driver.find_element_by_xpath("//ul[@class='a-pagination']/li[2]/a").click()
                    time.sleep(3)
                except : break 

In [7]:
len(ratings), len(title), len(review_text)

(83810, 83810, 83810)

In [8]:
#creating a dataframe

data = list(zip(title,review_text,ratings))
df = pd.DataFrame(data, columns = ["Review_title","Reiew_text","Ratings"])
df

Review_title  \
0                      Think twice before going for it   
1                                           Overpriced   
2      Build quality is not as good as shown in the Ad   
3        Speed and Visuals makes this a power machine!   
4                            Overpriced and unreliable   
...                                                ...   
83805    Vfm. Range of Archer c6 or A6 without mu-mimo   
83806            USB port is not supported for printer   
83807                  A best buy for that price range   
83808                                  Value For Money   
83809                                                    

                                              Reiew_text             Ratings  
0      \n  &nbsp;Pros<br>-----<br>- Very light weight...  2.0 out of 5 stars  
1      \n  Overpriced for this mediocre product with ...  2.0 out of 5 stars  
2      \n  Did not meet expectations:<br>Takes signif...  3.0 out of 5 stars  
3      \n  The speakers and sound quality are patheti...  3.0 out of 5 stars  
4      \n  For just i5 11th gen this laptop is way ov...  3.0 out of 5 stars  
...                                                  ...                 ...  
83805  \n  Same wifi signal range as c6 or the new A6...  4.0 out of 5 stars  
83806  \n  USB port is not supporting  printer so I a...  4.0 out of 5 stars  
83807  \n  Like : Speed , look and also better range ...  4.0 out of 5 stars  
83808  \n  Got it for 2K. Wifi signal in 2.4 Ghz is v...  4.0 out of 5 stars  
83809                                                                         

[83810 rows x 3 columns]

In [9]:
#saving into csv file

df.to_csv("amazon1_reviews.csv")

In [2]:
#Loading the flipkart reviews file

flpkrt = pd.read_csv("flipkrt1_reviews.csv")
flpkrt

Unnamed: 0               Review_Title  \
0               0          Worth every penny   
1               1                    Awesome   
2               2              Great product   
3               3                  Wonderful   
4               4       Good quality product   
...           ...                        ...   
30676       30676            Waste of money!   
30677       30677                       Nice   
30678       30678        Could be way better   
30679       30679                  Just wow!   
30680       30680  Did not meet expectations   

                                              Reiew_Text  Ratings  
0      Laptop is worth the price. It is fast and smoo...        5  
1                        Value for money just go for it✨        5  
2                                                   good        5  
3      It's good product but some times it gets stuck...        4  
4                       Great Laptop. You can go for it.        4  
...                                                  ...      ...  
30676                     It is worse screen was damaged        1  
30677                                               Good        3  
30678                                    Poor quality...        2  
30679                                       Good product        5  
30680                                           Very bad        1  

[30681 rows x 4 columns]

In [3]:
#Loading the amazon reviews file

df1 = pd.read_csv("amazon1_reviews.csv")
df1

Unnamed: 0                                     Review_title  \
0               0                  Think twice before going for it   
1               1                                       Overpriced   
2               2  Build quality is not as good as shown in the Ad   
3               3    Speed and Visuals makes this a power machine!   
4               4                        Overpriced and unreliable   
...           ...                                              ...   
83805       83805    Vfm. Range of Archer c6 or A6 without mu-mimo   
83806       83806            USB port is not supported for printer   
83807       83807                  A best buy for that price range   
83808       83808                                  Value For Money   
83809       83809                                              NaN   

                                              Reiew_text             Ratings  
0      \n  &nbsp;Pros<br>-----<br>- Very light weight...  2.0 out of 5 stars  
1      \n  Overpriced for this mediocre product with ...  2.0 out of 5 stars  
2      \n  Did not meet expectations:<br>Takes signif...  3.0 out of 5 stars  
3      \n  The speakers and sound quality are patheti...  3.0 out of 5 stars  
4      \n  For just i5 11th gen this laptop is way ov...  3.0 out of 5 stars  
...                                                  ...                 ...  
83805  \n  Same wifi signal range as c6 or the new A6...  4.0 out of 5 stars  
83806  \n  USB port is not supporting  printer so I a...  4.0 out of 5 stars  
83807  \n  Like : Speed , look and also better range ...  4.0 out of 5 stars  
83808  \n  Got it for 2K. Wifi signal in 2.4 Ghz is v...  4.0 out of 5 stars  
83809                                                NaN                 NaN  

[83810 rows x 4 columns]

### Updating the Column Names to same Names in both files

In [4]:
# Changing the column name in amazon file similar to flipkart file

df1.rename(columns = {'Review_title':'Review_Title'}, inplace = True)

In [5]:
# Changing the column name in amazon file similar to flipkart file

df1.rename(columns = {'Reiew_text':'Review_Text'}, inplace = True)

In [6]:
# Changing the column name in flipkart file

flpkrt.rename(columns = {'Reiew_Text':'Review_Text'}, inplace = True)

In [7]:
# Dropping Unnamed: 0 in flipkart file

flpkrt.drop(columns = 'Unnamed: 0', inplace=True)

In [8]:
# Dropping Unnamed: 0 in amazon file

df1.drop(columns = 'Unnamed: 0', inplace=True)

In [9]:
#Concating both files

Ratings_Prediction = pd.concat([df1, flpkrt],ignore_index=True)
Ratings_Prediction

Review_Title  \
0                       Think twice before going for it   
1                                            Overpriced   
2       Build quality is not as good as shown in the Ad   
3         Speed and Visuals makes this a power machine!   
4                             Overpriced and unreliable   
...                                                 ...   
114486                                  Waste of money!   
114487                                             Nice   
114488                              Could be way better   
114489                                        Just wow!   
114490                        Did not meet expectations   

                                              Review_Text             Ratings  
0       \n  &nbsp;Pros<br>-----<br>- Very light weight...  2.0 out of 5 stars  
1       \n  Overpriced for this mediocre product with ...  2.0 out of 5 stars  
2       \n  Did not meet expectations:<br>Takes signif...  3.0 out of 5 stars  
3       \n  The speakers and sound quality are patheti...  3.0 out of 5 stars  
4       \n  For just i5 11th gen this laptop is way ov...  3.0 out of 5 stars  
...                                                   ...                 ...  
114486                     It is worse screen was damaged                   1  
114487                                               Good                   3  
114488                                    Poor quality...                   2  
114489                                       Good product                   5  
114490                                           Very bad                   1  

[114491 rows x 3 columns]

In [10]:
Ratings_Prediction.to_csv("Ratings_Prediction_Scrapped_Data_File.csv")